In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# PARTE 1: DEFINICIÓN DE PARÁMETROS DEL PROTOTIPO Y GENERACIÓN DE DATOS
# ====================================================================

print("Iniciando Fase 1: Preparación de Datos y Cálculo del Baseline")

# --- PARÁMETROS DEL PROTOTIPO ---
HORIZONTE_DIAS = 3
CAPACIDAD_INSTALADA_SOLAR = 100.0  # MW
CAPACIDAD_RESTRINGIDA = 80.0      # MW (El cuello de botella crítico)
HORAS = 24 * HORIZONTE_DIAS

# 1. Generación de Series de Tiempo
index = pd.date_range(start='2024-06-01', periods=HORAS, freq='H')
df = pd.DataFrame(index=index)

# 2. Simulación de Demanda (Patrón diario típico)
demanda_base = 50  # MW base
demanda_pico = 40  # MW de pico
# Creación de un patrón sinusoidal para la demanda diaria
demanda_diaria = demanda_pico * np.sin(2 * np.pi * (df.index.hour - 8) / 24)
df['Demanda_MW'] = demanda_base + demanda_diaria + np.random.normal(0, 2, HORAS)
df['Demanda_MW'] = df['Demanda_MW'].clip(lower=30) # Mínimo realista

# 3. Simulación de Generación Solar (Con variabilidad y estrés intencional)
# Generamos una curva solar que claramente excede el límite en ciertos momentos
solar_generada = CAPACIDAD_INSTALADA_SOLAR * np.sin(2 * np.pi * (df.index.hour - 6) / 48) * 1.5
# Aplicamos un factor de "día de estrés" donde la generación es muy alta y constante entre las 10am y 4pm
estres_factor = np.where((df.index.hour > 10) & (df.index.hour < 16), 1.2, 1.0)

df['Generacion_Solar_MW'] = (solar_generada * 0.8 + np.random.normal(0, 5, HORAS)).clip(lower=0)
df['Generacion_Solar_MW'] = df['Generacion_Solar_MW'] * df.index.dayofyear/365 * 1.5 * estres_factor


# Visualización rápida del estrés
plt.figure(figsize=(14, 5))
plt.plot(df['Demanda_MW'], label='Demanda (Total)', color='gray', alpha=0.7)
plt.plot(df['Generacion_Solar_MW'], label=f'Solar Generada (Capacidad {CAPACIDAD_INSTALADA_SOLAR} MW)', color='orange')
plt.axhline(CAPACIDAD_RESTRINGIDA, color='red', linestyle='--', label=f'Límite de Transmisión ({CAPACIDAD_RESTRINGIDA} MW)')
plt.title('Estrés de Generación vs. Restricción de Transmisión (3 Días Simulados)')
plt.ylabel('Potencia (MW)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Máxima generación solar simulada: {df['Generacion_Solar_MW'].max():.2f} MW")


# ====================================================================
# PARTE 2: CÁLCULO DEL BASELINE (Gestión Ingenua)
# ====================================================================

# El operador solo puede inyectar hasta el límite de 80 MW.
# El exceso se desperdicia inmediatamente, ignorando cualquier recurso de almacenamiento.

df['Exceso_Solar'] = df['Generacion_Solar_MW'] - CAPACIDAD_RESTRINGIDA

# El desperdicio es el exceso, pero no puede ser negativo
df['Curtailment_Baseline_MW'] = df['Exceso_Solar'].apply(lambda x: max(0, x))

# Calculamos la energía total desperdiciada en el periodo
total_curtailment_baseline = df['Curtailment_Baseline_MW'].sum()

print(f"\n--- Resultados Baseline (Gestión Ingenua) ---")
print(f"Total de Horas en Estrés (Exceso > 0): {len(df[df['Exceso_Solar'] > 0])} horas")
print(f"TOTAL DE ENERGÍA DESPERDICIADA (MWh): {total_curtailment_baseline:.2f} MWh")

# Cálculo de la inyección real bajo el Baseline
df['Inyeccion_Baseline_MW'] = df['Generacion_Solar_MW'] - df['Curtailment_Baseline_MW']

# 5. Visualización de la inyección bajo Baseline
plt.figure(figsize=(14, 4))
plt.plot(df['Demanda_MW'], label='Demanda', color='gray', alpha=0.7)
plt.plot(df['Inyeccion_Baseline_MW'], label='Inyección (Baseline)', color='green')
plt.axhline(CAPACIDAD_RESTRINGIDA, color='red', linestyle='--', label='Límite de Transmisión')
plt.title('Inyección de Energía Bajo Política Baseline (Corte Inmediato)')
plt.ylabel('Potencia (MW)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

Iniciando Fase 1: Preparación de Datos y Cálculo del Baseline


/var/folders/1b/z4m03y610vv80mggpwr9zd400000gn/T/ipykernel_25329/3343862492.py:17: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  index = pd.date_range(start='2024-06-01', periods=HORAS, freq='H')


AttributeError: 'Index' object has no attribute 'clip'